# Fund Strategy - Factor Strategy 0001

## 1. Import Library

In [1]:
import dai
import random
import pandas as pd

## 2. Prepare Strategy Data

In [2]:
sd = '2022-12-31'
ed = '2026-02-13'

In [3]:
sql_factor = """
SELECT
    date,
    instrument,
    -1 * navg_rate AS factor
FROM cn_fund_bar1d JOIN cn_fund_nav USING (date, instrument)
WHERE instrument IN ('510170.SH', '513520.SH', '513030.SH', '159941.SZ', '518880.SH', '159920.SZ', '510300.SH', '511010.SH')
"""

In [4]:
sql_trade = f"""
WITH
data_strategy AS (
    {sql_factor}
),
data_filter AS (
    SELECT
        date,
        instrument,
        factor AS score,
        c_rank(factor) AS score_rank,
    FROM data_strategy
    QUALIFY score_rank <= 3
),
data_date AS (
    SELECT
        date,
        instrument,
        score, 
        score_rank, 
        1 / c_sum(1) AS position, 
    FROM data_filter JOIN mldt_cn_stock_calendar_daily USING (date)
    WHERE is_month_end_trade = 1
)
SELECT *
FROM data_date
ORDER BY date, score_rank
"""

In [5]:
df_strategy = dai.query(sql_trade, filters={"date":[sd, ed]}).df()
df_strategy

,date,instrument,score,score_rank,position
0,2023-01-31,159941.SZ,-0.0151,1.0,0.333333
1,2023-01-31,510170.SH,-0.0069,2.0,0.333333
2,2023-01-31,511010.SH,-0.0009,3.0,0.333333
3,2023-02-28,510300.SH,-0.0063,1.0,0.333333
4,2023-02-28,513030.SH,-0.0036,2.0,0.333333
...,...,...,...,...,...
106,2025-12-31,511010.SH,0.0001,2.0,0.333333
107,2025-12-31,510300.SH,0.0046,3.0,0.333333
108,2026-01-30,511010.SH,-0.0000,1.0,0.333333
109,2026-01-30,513520.SH,0.0066,2.0,0.333333


## 3. Backtest Strategy

In [6]:
from bigmodule import M

def BigTrader_Initialize(context):
    from bigtrader.finance.commission import PerOrder
    context.set_commission(PerOrder(buy_cost=0.0003, sell_cost=0.0013, min_cost=5))

def BigTrader_Before_Trading(context, data):
    pass

def BigTrader_Handle_Tick(context, tick):
    pass

def BigTrader_Handle_Data(context, data):

    df_now = context.data[context.data["date"] == data.current_dt.strftime("%Y-%m-%d")]

    if len(df_now) == 0:
        return
    
    target_instruments  = list(df_now["instrument"])
    holding_instruments = list(context.get_account_positions().keys())

    for instrument in holding_instruments:
        if instrument not in target_instruments:
            context.order_target_percent(instrument, 0)
        
    for i, x in df_now.iterrows():
        position = 0.0 if pd.isnull(x.position) else float(x.position)
        context.order_target_percent(x.instrument, position)

def BigTrader_Handle_Trade(context, trade):
    pass

def BigTrader_Handle_Order(context, order):
    pass

def BigTrader_After_Trading(context, data):
    pass

BigTrader = M.bigtrader.v34(
    
    data = df_strategy,
    
    start_date = """""",
    end_date   = """""",
    
    initialize           = BigTrader_Initialize,
    before_trading_start = BigTrader_Before_Trading,
    handle_tick          = BigTrader_Handle_Tick,
    handle_data          = BigTrader_Handle_Data,
    handle_trade         = BigTrader_Handle_Trade,
    handle_order         = BigTrader_Handle_Order,
    after_trading        = BigTrader_After_Trading,
    
    capital_base = 1000000  + random.uniform(0, 10),
    frequency="""daily""",
    product_type="""自动""",
    rebalance_period_type="""交易日""",
    rebalance_period_days="""1""",
    rebalance_period_roll_forward=True,
    backtest_engine_mode="""标准模式""",
    before_start_days=0,
    volume_limit=1,
    order_price_field_buy="""open""",
    order_price_field_sell="""open""",
    benchmark="""沪深300指数""",
    
    plot_charts=True,
    debug=False,
    backtest_only=False,
    m_name="""BigTrader"""
) 

[2026-02-19 14:40:21] [info     ] bigtrader.v34 开始运行 ..
[2026-02-19 14:40:21] [info     ] 2023-01-31, 2026-01-30, , , instruments=8
[2026-02-19 14:40:21] [info     ] bigtrader module V2.2.0
[2026-02-19 14:40:21] [info     ] bigtrader engine v0.1.0.post9+g7a244b6 2026-02-10
[2026-02-19 14:40:22] [info     ] backtest done, raw_perf_ds:dai.DataSource("_1d8a7dbd143d47c09fdb94674ce68c7c")


[2026-02-19 14:40:23] [info     ] bigtrader.v34 运行完成 [1.994s].
